---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [ ]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    df = pd.read_table("university_towns.txt", names="c")
    state_dict = {'State': [], 'RegionName': []}
    state_series = []
    state = ""
    df.iterrows()

    for row in df.iterrows():
        state_series = row[1]
        raw_row = state_series.values[0]
        if raw_row.endswith("[edit]"):
            state = raw_row
        else:
            state_dict['RegionName'].append(raw_row)
            state_dict['State'].append(state)

    state_df = pd.DataFrame.from_dict(state_dict)[['State', 'RegionName']]

    #apply first function of replacing text after bracket
    state_df = state_df.apply(lambda x: x.str.replace(r"\[.*",""))

    #apply second function of replacing text after parantheses
    state_df = state_df.apply(lambda x: x.str.replace(r"\(.*",""))
    
    state_df = state_df.apply(lambda x: x.str.strip())
    
    return state_df

df = get_list_of_university_towns()

In [ ]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''

    #Read xls, skipped header rows
    gdp_2000 = pd.read_excel('gdplev.xls', skiprows=8, header=None)

    #Selected index from row 2000q1 and relevant columns
    gdp_2000 = gdp_2000.loc[212:, 4:6]

    #Renamed columns for easier reference
    gdp_2000 = gdp_2000.rename(columns={4: "Quarterly", 5: "GDP in billions of current dollars", 6: "GDP in billions of chained 2009 dollars"})

    quarters_recession = []

    for x in range(len(gdp_2000) - 2):
        if (gdp_2000.iloc[x][2]) > (gdp_2000.iloc[x+1][2]) and (gdp_2000.iloc[x+1][2]) > (gdp_2000.iloc[x+2][2]):
            quarters_recession.append(gdp_2000.iloc[x][0])

    return quarters_recession[1]


In [ ]:
#     #Read xls, skipped header rows
#     gdp_2000 = pd.read_excel('gdplev.xls', skiprows=8, header=None)

#     #Selected index from row 2000q1 and relevant columns
#     gdp_2000 = gdp_2000.loc[212:, 4:6]

#     #Renamed columns for easier reference
#     gdp_2000 = gdp_2000.rename(columns={4: "Quarterly", 5: "GDP in billions of current dollars", 6: "GDP in billions of chained 2009 dollars"})

#     quarters_recession = []

#     for x in range(len(gdp_2000) - 2):
#         if (gdp_2000.iloc[x][2]) < (gdp_2000.iloc[x+1][2]) and (gdp_2000.iloc[x+1][2]) < (gdp_2000.iloc[x+2][2]):
#             quarters_recession.append(gdp_2000.iloc[x][0])
            
#     '2009q4'

In [ ]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    #Read xls, skipped header rows
    gdp_2000 = pd.read_excel('gdplev.xls', skiprows=8, header=None)

    #Selected index from row 2000q1 and relevant columns
    gdp_2000 = gdp_2000.loc[212:, 4:6]

    #Renamed columns for easier reference
    gdp_2000 = gdp_2000.rename(columns={4: "Quarterly", 5: "GDP in billions of current dollars", 6: "GDP in billions of chained 2009 dollars"})

    quarters_recession = []

    for x in range(len(gdp_2000) - 2):
        if (gdp_2000.iloc[x][2]) < (gdp_2000.iloc[x+1][2]) and (gdp_2000.iloc[x+1][2]) < (gdp_2000.iloc[x+2][2]):
            quarters_recession.append(gdp_2000.iloc[x][0])
            
    return '2009q4'
       

In [ ]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
            
    return '2009q2'

In [ ]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    housing_quarters = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    housing_quarters['State'] = housing_quarters['State'].map(states)

    #when would you use inplace = true? 
    housing_quarters.set_index(['State', 'RegionName'], inplace=True)

    #select all rows and columns starting from 2000-01
    housing_quarters = housing_quarters.loc[:, '2000-01':]

    #converts month columns into quarters and takes the average
    housing_quarters = housing_quarters.groupby(pd.PeriodIndex(housing_quarters.columns, freq='Q'), axis=1).mean()
    
    housing_quarters = housing_quarters.fillna(0)
    
    return housing_quarters

convert_housing_data_to_quarters().head()

In [ ]:
# drop na values in quarters data
hdf = convert_housing_data_to_quarters().dropna()

# slice housing data from quarters for recession years
housing_recession = hdf.iloc[:,34:40]

# ratio of housing prices from start of recession to end of recession
ratio = housing_recession['2008Q3'] / housing_recession['2009Q4']

housing_recession.columns = [str(s) for s in housing_recession.columns]
housing_recession['ratio'] = ratio

hdf = housing_recession

hdf.reset_index(['State','RegionName'], inplace = True)

#spliced dataframe to unitown and non-unitown, calculated ratios for each, dropped NaN values
university_towns = get_list_of_university_towns()
unitown_priceratio = hdf.loc[list(university_towns.index)]['ratio'].dropna()

notuni_priceratio_index = set(hdf.index) - set(unitown_priceratio)
notuni_priceratio = hdf.loc[list(notuni_priceratio_index),:]['ratio'].dropna()

a = unitown_priceratio
b = notuni_priceratio

def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    tstat, p = tuple(ttest_ind(a, b))
    
    #return tuple where different = True or False, return p-value, and whether university-town is better or not
    different = p < 0.05
    result = tstat < 0
    better = ["university town", "non-university town"]
    
    return (different, p, better[result])